<a href="https://colab.research.google.com/github/moatazomar2311/covid_19_Xray/blob/main/covid_19_Xray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: import #!/bin/bash
# kaggle datasets download alifrahman/covid19-chest-xray-image-dataset

!pip install kaggle
# Copy kaggle.json to the appropriate location
!mkdir -p ~/.kaggle
!cp /content/drive/My\ Drive/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset
!kaggle datasets download alifrahman/covid19-chest-xray-image-dataset


cp: cannot stat '/content/drive/My Drive/kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/alifrahman/covid19-chest-xray-image-dataset
License(s): Attribution 4.0 International (CC BY 4.0)
 71% 29.0M/40.6M [00:00<00:00, 66.3MB/s]
100% 40.6M/40.6M [00:00<00:00, 84.9MB/s]


In [ ]:
!unzip /content/covid19-chest-xray-image-dataset.zip

Archive:  /content/covid19-chest-xray-image-dataset.zip
  inflating: dataset/covid/01E392EE-69F9-4E33-BFCE-E5C968654078.jpeg  
  inflating: dataset/covid/1-s2.0-S0140673620303706-fx1_lrg.jpg  
  inflating: dataset/covid/1-s2.0-S0929664620300449-gr2_lrg-a.jpg  
  inflating: dataset/covid/1-s2.0-S0929664620300449-gr2_lrg-b.jpg  
  inflating: dataset/covid/1-s2.0-S0929664620300449-gr2_lrg-c.jpg  
  inflating: dataset/covid/1-s2.0-S0929664620300449-gr2_lrg-d.jpg  
  inflating: dataset/covid/1-s2.0-S1684118220300608-main.pdf-001.jpg  
  inflating: dataset/covid/1-s2.0-S1684118220300608-main.pdf-002.jpg  
  inflating: dataset/covid/1-s2.0-S1684118220300682-main.pdf-002-a1.png  
  inflating: dataset/covid/1-s2.0-S1684118220300682-main.pdf-002-a2.png  
  inflating: dataset/covid/1-s2.0-S1684118220300682-main.pdf-003-b1.png  
  inflating: dataset/covid/1-s2.0-S1684118220300682-main.pdf-003-b2.png  
  inflating: dataset/covid/1312A392-67A3-4EBF-9319-810CF6DA5EF6.jpeg  
  inflating: dataset/covid

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import datetime
import numpy as np
import os
from sklearn.metrics import classification_report

# Constants
INIT_LR = 1e-4
EPOCHS = 25
BS = 32
DIRECTORY = '/content/dataset'
CATEGORIES = ["covid", "normal"]

# Data Augmentation and Preprocessing
aug_train = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest",
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    validation_split=0.2
)

aug_val = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    validation_split=0.2
)

# Data Generators
train_generator = aug_train.flow_from_directory(
    DIRECTORY,
    class_mode="categorical",
    target_size=(300, 300),  # EfficientNetB3 uses 300x300 images
    shuffle=True,
    subset="training",
    batch_size=BS
)

val_generator = aug_val.flow_from_directory(
    DIRECTORY,
    class_mode="categorical",
    target_size=(300, 300),
    shuffle=False,
    subset="validation",
    batch_size=BS
)

# Model Architecture (EfficientNetB3)
baseModel = EfficientNetB3(weights="imagenet", include_top=False, input_tensor=Input(shape=(300, 300, 3)))

headModel = baseModel.output
headModel = GlobalAveragePooling2D()(headModel)
headModel = Flatten()(headModel)
headModel = Dense(256, activation="relu")(headModel)  # Increase the number of neurons
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

# Combine base model and head
model = Model(inputs=baseModel.input, outputs=headModel)

# Fine-tune the entire model
for layer in baseModel.layers[-100:]:
    layer.trainable = True

# Compile the model
opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

# Callbacks
checkpoint = ModelCheckpoint(
    filepath="best_model.h5",
    monitor="val_loss",  # Monitor validation loss
    save_best_only=True,
    mode="min",
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=2,
    min_lr=1e-6,
    verbose=1
)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=(train_generator.samples // BS),
    validation_data=val_generator,
    validation_steps=(val_generator.samples // BS),
    epochs=EPOCHS,
    callbacks=[
        EarlyStopping(monitor="accuracy", patience=3, restore_best_weights=True),
        checkpoint,
        reduce_lr
    ]
)

# Evaluate the model
predictions = np.argmax(model.predict(val_generator), axis=1)

print(classification_report(val_generator.classes, predictions, target_names=CATEGORIES))


Found 76 images belonging to 2 classes.
Found 18 images belonging to 2 classes.
43941136/43941136 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(None, 300, 300, 3))
  warnings.warn(msg)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 69s/step - accuracy: 0.6856 - loss: 0.6307   
Epoch 1: val_loss improved from inf to 0.56330, saving model to best_model.h5


...

2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 1.0000 - loss: 0.0190 - val_accuracy: 1.0000 - val_loss: 0.1026 - learning_rate: 1.0000e-04
Epoch 24/25
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 472ms/step - accuracy: 1.0000 - loss: 0.0120
Epoch 24: val_loss improved from 0.10264 to 0.09981, saving model to best_model.h5


2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 1.0000 - loss: 0.0120 - val_accuracy: 1.0000 - val_loss: 0.0998 - learning_rate: 1.0000e-04
Epoch 25/25
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 436ms/step - accuracy: 1.0000 - loss: 0.0145
Epoch 25: val_loss improved from 0.09981 to 0.09246, saving model to best_model.h5


2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 3s/step - accuracy: 1.0000 - loss: 0.0148 - val_accuracy: 1.0000 - val_loss: 0.0925 - learning_rate: 1.0000e-04


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(18, 300, 300, 3))
  warnings.warn(msg)


1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step
              precision    recall  f1-score   support

       covid       1.00      1.00      1.00        13
      normal       1.00      1.00      1.00         5

    accuracy                           1.00        18
   macro avg       1.00      1.00      1.00        18
weighted avg       1.00      1.00      1.00        18



In [ ]:

# Evaluate the model
loss, accuracy = model.evaluate(val_generator)
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 677ms/step - accuracy: 1.0000 - loss: 0.0925
Loss: 0.09246048331260681
Accuracy: 1.0
